# 1. EDA

Tổng cộng, được cung cấp 7 tệp.

>Điều chỉnh giáo dục phù hợp với trình độ khả năng của học sinh là một trong nhiều điều có giá trị mà một gia sư AI có thể làm. Thử thách của bạn trong cuộc thi này là một phiên bản của nhiệm vụ tổng thể đó; bạn sẽ dự đoán liệu học sinh có thể trả lời chính xác các câu hỏi tiếp theo của họ hay không. Bạn sẽ được cung cấp cùng một loại thông tin mà một ứng dụng giáo dục hoàn chỉnh sẽ có: thành tích lịch sử của học sinh đó, thành tích của các học sinh khác trong cùng một câu hỏi, siêu dữ liệu về chính câu hỏi đó, v.v.

>Đây là cuộc thi viết mã theo chuỗi thời gian, bạn sẽ nhận được dữ liệu bộ thử nghiệm và đưa ra dự đoán với API chuỗi thời gian của Kaggle. Hãy đảm bảo xem xét kỹ phần Chi tiết API chuỗi thời gian.

Tại đây cài đặt và tải các thư viện của mình và đặt một số cài đặt mặc định để sử dụng trong tương lai.

In [ ]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null

In [ ]:
# Nhập các thư viện cơ bản cho EDA:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#

import datatable as dt # để tải .csv nhanh hơn

#

import gc # để xóa bộ nhớ
import warnings # để ẩn cảnh báo
warnings.filterwarnings('ignore')

In [ ]:
# Cài đặt tạo kiểu:

plt.rcParams['figure.figsize'] = [18,10]
plt.style.use('ggplot')

# Tải dữ liệu

Dữ liệu đưa ra là rất lớn, không dễ dàng để tải tất cả chúng vào RAM mà không bị hết bộ nhớ. Cần đặt các kiểu dtypes cho mỗi cột để giảm mức sử dụng bộ nhớ. Theo mặc định, chúng là 32/64 cho các cột số, nhưng có thể chọn các loại theo cách thủ công dựa trên số lượng tối đa của chúng trong cột. Các lựa chọn kiểu loại này dựa trên:

> int8 / uint8: sử dụng 1 byte bộ nhớ, phạm vi từ -128/127 hoặc 0/255,

> bool: sử dụng 1 byte, true hoặc false,

> float16 / int16 / uint16: sử dụng 2 byte bộ nhớ, phạm vi từ -32768 đến 32767 hoặc 0/65535,

> float32 / int32 / uint32: sử dụng 4 byte bộ nhớ, phạm vi từ -2147483648 đến 2147483647,

> float64 / int64 / uint64: sử dụng 8 byte bộ nhớ.

[Source](https://medium.com/@vincentteyssier/optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment-5f07db3d72e)

Sử dụng ** datatable ** để tải nhanh hơn. Bạn có thể tìm hiểu giải thích sâu hơn ở [here](https://www.kaggle.com/rohanrao/tutorial-on-reading-large-datasets).



# Exploring Train

In [ ]:
# Dict for dtypes:

data_types = {
    'row_id': 'int32',
    'timestamp': 'int64',
    'user_id': 'int64',
    'content_id': 'int16',
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'
}

In [ ]:
# Tải dữ liệu với datatable và chuyển đổi nó thành pandas df:

train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv').to_pandas()

# Chọn ngẫu nhiên một phần dữ liệu để xử lý nhanh hơn.

train_df = train_df.sample(len(train_df)//5,random_state=42)


# Đặt loại dtypes cho mỗi cột

for column, d_type in data_types.items():
    train_df[column] = train_df[column].astype(d_type) 

In [ ]:
# Tải các tệp dữ liệu khác:

questions_df = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
lectures_df = pd.read_csv('../input/riiid-test-answer-prediction/lectures.csv')
test_df = pd.read_csv('../input/riiid-test-answer-prediction/example_test.csv')

In [ ]:
# Hiển thị các loại và sử dụng bộ nhớ.

train_df.info()

## Timestamp

#### "timestamp": Thời gian tính bằng mili giây giữa lần tương tác của người dùng này đến khi hoàn thành sự kiện đầu tiên từ người dùng đó.

#### Tại đây, có thể thấy các phần trước đó của dòng thời gian hoạt động nhiều hơn so với các phiên dài hơn, điều này được mong đợi. Ngoài ra, có thể nhận thấy rằng có một số người dùng dành thời gian khá dài!

In [ ]:
# Vẽ đồ thị liên quan đến timestamp

fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(32,14))

sns.distplot(train_df.timestamp, kde=False,hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'alpha': 0.8}, bins=100, ax=ax[0])

ax[0].set_xlabel('Time in Miliseconds')
ax[0].set_ylabel('Count')
ax[0].set_title('Timestamp Distribution', weight='bold')


sns.distplot(train_df.groupby('user_id').agg({'timestamp': 'mean'}), kde=False, hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'alpha': 0.8}, bins=50,ax=ax[1])

ax[1].set_xlabel('Time in Miliseconds*')
ax[1].set_ylabel('Count')
ax[1].set_title('Mean Timestamp Per User Distribution', weight='bold')

plt.show()

# Người dùng và Nội dung

##### Ở đây có ID duy nhất cho mỗi người dùng, nếu tính tất cả các mối quan tâm do người dùng thực hiện, có thể thấy có một số người dùng khá tích cực, trong số ~ 300 nghìn người dùng duy nhất, chúng tôi thấy 25 người hàng đầu gần như kiếm được hơn 3 nghìn các tương tác.

#### Nội dung khá giống với ID người dùng. Tại đây, có thể thấy hầu hết các nội dung phổ biến, có vẻ như nội dung # 6116 thực sự được yêu thích nhất, tiếp theo là # 6173 và # 4120 với khoảng 400 nghìn lượt tương tác.

In [ ]:
# Vẽ biểu đồ liên quan đến người dùng và nội dung.

fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(32,14))

# Distplot:

sns.countplot(y='user_id', data=train_df, order=train_df.user_id.value_counts().index[:25], palette='autumn',ax = ax[0])
ax[0].set_title('Top 25 Active Users', weight='bold')

# Countplot:

sns.countplot(y='content_id', data=train_df, order=train_df.content_id.value_counts().index[:25], palette='autumn',ax = ax[1])
ax[1].set_title('Top 25 Content', weight='bold')


plt.show()

# Loại nội dung

#### "content_type_id": 0 nếu sự kiện là một câu hỏi được đặt ra cho người dùng, 1 nếu sự kiện là người dùng đang xem một bài giảng.

#### Ở đây chúng ta có thể thấy rằng 98% mẫu của chúng ta là câu hỏi và ~ 2% là bài giảng.

In [ ]:
# Các loại nội dung vẽ sơ đồ

g=sns.countplot(train_df.content_type_id, palette='autumn')

# Thêm phần trăm

total = float(len(train_df['content_type_id']))

for p in g.patches:
    height = p.get_height()
    g.text(p.get_x() + p.get_width() / 2.,
            height + 2,
            '{:1.2f}%'.format((height / total) * 100),
            ha='center')

plt.ylabel('Count*10^7')    
plt.title('Content Types - 0: Question, 1: Lecture', weight='bold')
plt.show()

# Vùng chứa Tác vụ

#### "task_container_id": Mã id cho loạt câu hỏi hoặc bài giảng. Ví dụ: một người dùng có thể thấy ba câu hỏi liên tiếp trước khi xem giải thích cho bất kỳ câu hỏi nào trong số đó. Tất cả ba câu đó sẽ chia sẻ một task_container_id.

#### Có thể thấy rằng các tác vụ có ID nhỏ hơn phổ biến hơn nhiều so với các số lớn hơn, trong khi đó, nhiệm vụ phổ biến nhất là # 14

In [ ]:
# Lập đồ thị vùng chứa nhiệm vụ:

fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(32,14))

# Phân bổ:

sns.distplot(train_df.task_container_id, kde=False,hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'alpha': 0.8}, ax=ax[0])


ax[0].set_ylabel('Frequency')
ax[0].set_title('Task Container ID Distribution', weight='bold')

# Số lượng:

sns.countplot(y='task_container_id', data=train_df, order=train_df.task_container_id.value_counts().index[:25], palette='autumn', ax=ax[1])
ax[1].set_title('Top 25 Tasks', weight='bold')


plt.show()

# Câu trả lời của người dùng

#### "user_answer": Câu trả lời của người dùng cho câu hỏi, nếu có. Đọc -1 là null cho các bài giảng.

#### Ở đây có thể thấy tùy chọn trả lời số 2 ít phổ biến hơn so với các câu trả lời còn lại trong ba câu trả lời. Có vẻ như người dùng / người hướng dẫn không thích tùy chọn số 2 cho lắm :)

In [ ]:
# Lập đồ thị câu trả lời của người dùng:

g=sns.countplot(train_df.user_answer, hue=train_df.answered_correctly, palette='autumn', order=train_df.user_answer.value_counts().index)

# Thêm phần trăm:

total = float(len(train_df['user_answer']))

for p in g.patches:
    height = p.get_height()
    g.text(p.get_x() + p.get_width() / 2.,
            height + 2,
            '{:1.2f}%'.format((height / total) * 100),
            ha='center')

plt.title('False/Correct per User Answer  (-1 for Lectures)', weight='bold')

plt.show()

# Câu hỏi trước_Prior Questions


#### before_question_elapsed_time: Thời gian trung bình tính bằng mili giây người dùng trả lời từng câu hỏi trong gói câu hỏi trước đó, bỏ qua bất kỳ bài giảng nào ở giữa. Không có giá trị cho gói câu hỏi hoặc bài giảng đầu tiên của người dùng. Lưu ý rằng thời gian là thời gian trung bình mà người dùng dành để giải quyết từng câu hỏi trong nhóm trước đó.

#### before_question_had_explanation: Người dùng có thấy giải thích và (các) câu trả lời chính xác hay không sau khi trả lời gói câu hỏi trước đó, bỏ qua bất kỳ bài giảng nào ở giữa. Giá trị được chia sẻ trên một gói câu hỏi và không có giá trị đối với gói câu hỏi hoặc bài giảng đầu tiên của người dùng. Thông thường, một số câu hỏi đầu tiên mà người dùng nhìn thấy là một phần của kiểm tra chẩn đoán giới thiệu mà họ không nhận được bất kỳ phản hồi nào.

In [ ]:
# Vẽ đồ thị cho các câu hỏi trước những nội dung liên quan:

fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(32,14))


sns.distplot(train_df.prior_question_elapsed_time.dropna(), kde=False, hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'alpha': 0.8}, ax=ax[0])

ax[0].set_ylabel('Count')
ax[0].set_title('Prior Question Elapsed Time Distribution', weight='bold')


g=sns.countplot(train_df.prior_question_had_explanation.dropna(), palette='autumn', ax=ax[1])

# thêm phần trăm

total = float(len(train_df.prior_question_had_explanation.dropna()))
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x() + p.get_width() / 2.,
            height + 2,
            '{:1.2f}%'.format((height / total) * 100),
            ha='center')

ax[1].set_title('Prior Question Elapsed had Explanation?', weight='bold')
    
plt.show()

In [ ]:
# Bỏ các bài giảng khỏi khung dữ liệu

train_df = train_df.loc[train_df['answered_correctly'] != -1].reset_index(drop=True)

# Câu hỏi

#### Tại đây, chúng tôi thêm dữ liệu bổ sung mà đã cung cấp. Vì vậy, có thể tìm thấy một số gợi ý sâu sắc ...

In [ ]:
# Hợp nhất dữ liệu câu hỏi với dữ liệu train:

train_df = pd.merge(train_df,questions_df[['question_id','part']], how='left', left_on='content_id', right_on='question_id').sort_values('row_id')
train_df['part'] = train_df['part'].astype('int8')


# Phần Câu hỏi

#### part: Phần liên quan của bài thi TOEIC.

#### Hợp nhất các phần câu hỏi dựa trên ID câu hỏi cụ thể của chúng. Những  phần liên quan của bài thi TOEIC được giải thích ở đây:

![](https://i.imgur.com/2wqNAJ1.png)
![](https://i.imgur.com/4B3AQyL.png)

### Ở đây chúng tôi nhận thấy rằng các câu hỏi Phần 5 là câu hỏi phổ biến nhất, trong đó phải hoàn thành các câu bằng bốn lựa chọn cho sẵn.

In [ ]:
g=sns.countplot(train_df.part, hue=train_df.answered_correctly, palette='autumn')

# thêm phần trăm

total = float(len(train_df.part))
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x() + p.get_width() / 2.,
            height + 2,
            '{:1.2f}%'.format((height / total) * 100),
            ha='center')

plt.title('False/Correct per Question Part', weight='bold')
plt.show()

In [ ]:
# nhóm user id và lấy trung bình, tổng, số

usr_ans = train_df.groupby('user_id').agg({ 'answered_correctly': ['mean','sum', 'count']})
usr_ans.columns = ['avg_correct_answer','num_of_correct', 'total_answers']

# thay đổi loại để giảm bộ nhớ (mặc định = 64)

usr_ans['num_of_correct'] = usr_ans['num_of_correct'].astype('int16')
usr_ans['total_answers'] = usr_ans['total_answers'].astype('int16')


train_df = pd.merge(train_df, usr_ans, how='left', on = 'user_id')

# Độ chính xác của câu trả lời đúng

#### lọc những người dùng đã trả lời hơn 100 câu hỏi và sắp xếp chúng dựa trên tỷ lệ câu trả lời đúng của họ, có một số người dùng khá chính xác!

In [ ]:
#  Biểu đồ 25 người dùng chính xác hàng đầu

sns.barplot(x='avg_correct_answer',y='user_id', orient='h', data=usr_ans[usr_ans['total_answers']>100].sort_values('avg_correct_answer', ascending=False).reset_index().iloc[:25],
            palette='autumn', order=usr_ans[usr_ans['total_answers']>100].sort_values('avg_correct_answer', ascending=False).reset_index().user_id.iloc[:25])

plt.title('Top 25 Accurate Users', weight='bold')
plt.show()

# Độ chính xác của câu trả lời so với Tổng số câu hỏi đã được trả lời

#### Có thể quan sát thấy tỷ lệ câu trả lời đúng ngày càng tăng trên tổng số câu hỏi mà người dùng trả lời. 

In [ ]:
# vẽ tổng số câu trả lời so với độ chính xác trung bình

sns.regplot(data=usr_ans[usr_ans['total_answers']> 100], y='avg_correct_answer', x='total_answers', ci=False, scatter_kws={'alpha':0.5}, line_kws={"color": "orange"})
plt.axhline(train_df.avg_correct_answer.mean(), color='k', linestyle='dashed', linewidth=3)
plt.axvline(train_df.total_answers.mean(), color='k', linestyle='dashed', linewidth=3)

min_ylim, max_ylim = plt.ylim()
plt.text(train_df.total_answers.mean()+25, max_ylim*0.20, 'Average Questions Solved {:.2f}'.format(train_df.total_answers.mean()))
plt.text(train_df.total_answers.mean()+2400, max_ylim*0.6, 'Average Correct Answer: {:.2f}'.format(train_df.avg_correct_answer.mean()))

plt.title('Average Correct Answer Ratio vs. Total Questions Answered per User', weight='bold')
plt.show()

## Độ chính xác của câu trả lời - Mối quan hệ về thời gian

#### Lấy thời gian tối đa của người dùng và lọc ra những người dùng nếu họ có ít hơn một giờ. Nhận được sự gia tăng nhẹ về độ chính xác của người dùng bằng cách tăng thời gian sử dụng nhưng có vẻ như không đáng kể.

In [ ]:
# Biểu đồ answer accuracy vs time

total_time = train_df.groupby('user_id')["timestamp"].max()
total_time = pd.merge(total_time.reset_index(), usr_ans.reset_index(), how='left', on = 'user_id')

sns.regplot(data=total_time[total_time['timestamp']> 3.6e+6], y='avg_correct_answer', x='timestamp', ci=False, scatter_kws={'alpha':0.5}, line_kws={"color": "orange"})

plt.axvline(total_time.timestamp.mean(), color='k', linestyle='dashed', linewidth=3)
plt.text(total_time.timestamp.mean()+total_time.timestamp.mean()*0.1, max_ylim*0.03, 'Average Time {:.2f}'.format(total_time.timestamp.mean()))

plt.title('Average Correct Answer Ratio vs. Time Spent', weight='bold')
plt.show()

# xóa một số biến để tiết kiệm bộ nhớ:

del total_time

gc.collect()

# Độ chính xác của câu trả lời - Mối quan hệ nội dung

#### Khi tính độ chính xác của câu trả lời theo nội dung, có thể thấy rằng nội dung phổ biến / chung chung hơn có tỷ lệ câu trả lời đúng thấp hơn. Trong khi đó các câu hỏi ít phổ biến / cụ thể hơn có tỷ lệ đúng cao hơn.

In [ ]:
# tạo tính năng mới dựa trên id nội dung

cnt_ans = train_df.groupby('content_id').agg({ 'answered_correctly': ['mean','sum', 'count']})
cnt_ans.columns = ['avg_correct_answer_c','num_of_correct_c', 'total_answers_c']

# thay đổi loại để giảm bộ nhớ (mặc định = 64)

cnt_ans['num_of_correct_c'] = cnt_ans['num_of_correct_c'].astype('int32')
cnt_ans['total_answers_c'] = cnt_ans['total_answers_c'].astype('int32')

train_df = pd.merge(train_df, cnt_ans, how='left', on = 'content_id')

In [ ]:
# Biểu đồ contents vs. answer accuracies

sns.regplot(data=cnt_ans[cnt_ans['total_answers_c']> 100], y='avg_correct_answer_c', x='total_answers_c', ci=False, scatter_kws={'alpha':0.5}, line_kws={"color": "orange"})

# Biểu đồ mean lines

plt.axhline(train_df.avg_correct_answer_c.mean(), color='k', linestyle='dashed', linewidth=3)
plt.axvline(train_df.total_answers_c.mean(), color='k', linestyle='dashed', linewidth=3)


min_ylim, max_ylim = plt.ylim()
plt.text(35000, max_ylim*0.65, 'Average Correct Answer: {:.2f}'.format(train_df.avg_correct_answer_c.mean()))
plt.text(5500, max_ylim*0.10, 'Average Questions Solved per Content: {:.2f}'.format(train_df.total_answers_c.mean()))

plt.title('Average Correct Answer Ratio vs. Total Questions Answered per Content', weight='bold')
plt.show()

In [ ]:
train_df.head()

# 2. Baseline Model

#### Đây là phần sẽ thực hiện một số mô hình cơ sở đơn giản để đo điểm chuẩn cho các mô hình tương lai của chúng tôi. Bắt đầu bằng cách điền vào một số giá trị còn thiếu trong dữ liệu và sau đó chia nó thành X và y để lập mô hình. Cần dự đoán xem người dùng đã trả lời câu hỏi cụ thể có đúng hay không.

In [ ]:
#Tạo biến X để training

X = train_df.copy()

In [ ]:
# điền giá trị N/A

X['prior_question_elapsed_time'].fillna(0,  inplace=True)
X['prior_question_had_explanation'] = X['prior_question_had_explanation'].fillna(value = False).astype(bool)

In [ ]:
# Xóa df train để làm sạch bộ nhớ:

del train_df

gc.collect()

In [ ]:
# Đặt X và Y để đào tạo:

X=X.sort_values(['user_id'])
y = X[["answered_correctly"]]
X = X.drop(["answered_correctly"], axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Nhận nhãn số cho dữ liệu phân loại:

lb_make = LabelEncoder()
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])
X['prior_question_had_explanation_enc'] = X['prior_question_had_explanation_enc'].astype('int8')
X.head()

In [ ]:
# Lựa chọn các tính năng để đào tạo:

X = X[['avg_correct_answer','num_of_correct','total_answers', 'avg_correct_answer_c', 'prior_question_elapsed_time','prior_question_had_explanation_enc','part']] 

In [ ]:
# Kiểm tra hình dạng của dữ liệu đào tạo để chắc chắn:
X.shape

### Dùng thuật toán phân loại Lightgbm tham số mặc định để training.

In [ ]:
# Tải (các) mô hình để thử nghiệm:
from sklearn.model_selection import StratifiedKFold, cross_validate
import lightgbm as lgb

light = lgb.LGBMClassifier(
)

# Validation

#### Phân tầng và xáo trộn mục tiêu  và xác thực nó bằng cách sử dụng 3 lần.

In [ ]:
# Đặt kfold phân tầng để xác thực:

kf = StratifiedKFold(3, shuffle=True, random_state=42)
classifiers = [light]

In [ ]:
def model_check(X, y, classifiers, cv):
    
    ''' Một chức năng để kiểm tra nhiều bộ phân loại và trả về một số số liệu. '''
    
    model_table = pd.DataFrame()

    row_index = 0
    for cls in classifiers:

        MLA_name = cls.__class__.__name__
        model_table.loc[row_index, 'Model Name'] = MLA_name
        
        cv_results = cross_validate(
            cls,
            X,
            y,
            cv=cv,
            scoring=('accuracy','f1','roc_auc'),
            return_train_score=True,
            n_jobs=-1
        )
        model_table.loc[row_index, 'Train Roc/AUC Mean'] = cv_results[
            'train_roc_auc'].mean()
        model_table.loc[row_index, 'Test Roc/AUC Mean'] = cv_results[
            'test_roc_auc'].mean()
        model_table.loc[row_index, 'Test Roc/AUC Std'] = cv_results['test_roc_auc'].std()

        model_table.loc[row_index, 'Time'] = cv_results['fit_time'].mean()

        row_index += 1        

    model_table.sort_values(by=['Test Roc/AUC Mean'],
                            ascending=False,
                            inplace=True)

    return model_table

# Results


In [ ]:
#Hiển thị kết quả mô hình mặc định:

raw_models = model_check(X, y, classifiers, kf)
display(raw_models)

# Prediction

#### Sử dụng môi trường dự đoán cạnh tranh cụ thể, dự đoán các mẫu thử nghiệm và gửi chúng bằng cách sử dụng gói 'riiideducation'.

In [ ]:
# Nhập gói riid:

import riiideducation

env = riiideducation.make_env()

iter_test = env.iter_test()

In [ ]:
# Fit mô hình

light.fit(X, y)

# Đặt lại các chỉ mục để hợp nhất trước:

cnt_ans=cnt_ans.reset_index()
usr_ans=usr_ans.reset_index()

In [ ]:
# Tải và dự đoán các mẫu thử nghiệm

for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.merge(usr_ans, how = 'left', on = 'user_id')
    test_df = test_df.merge(cnt_ans, how = 'left', on = 'content_id')
    test_df = pd.merge_ordered(test_df,questions_df[['question_id','part']], how='left', left_on='content_id', right_on='question_id', fill_method='ffill')
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value = False).astype(bool)
    test_df['prior_question_elapsed_time'].fillna(0,  inplace=True)
    test_df['avg_correct_answer'].fillna(0.5, inplace=True)
    test_df['avg_correct_answer_c'].fillna(0.5, inplace=True)
    test_df.fillna(value = -1, inplace = True)
    test_df["prior_question_had_explanation_enc"] = lb_make.fit_transform(test_df["prior_question_had_explanation"])
    
    
    y_pred = light.predict_proba(test_df[['avg_correct_answer','num_of_correct','total_answers', 'avg_correct_answer_c', 'prior_question_elapsed_time','prior_question_had_explanation_enc','part']])[:,1]
    test_df['answered_correctly'] = y_pred
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
